In [ ]:
import asyncio
import dash
import dash_core_components as dcc
import dash_html_components as html
from bleak import BleakScanner

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Сканирование Bluetooth устройств'),
    html.Label('Начальный номер устройства:'),
    dcc.Input(id='start-range', type='number', value=1),
    html.Label('Конечный номер устройства:'),
    dcc.Input(id='end-range', type='number', value=10),
    html.Label('Время сканирования (в секундах):'),
    dcc.Input(id='scan-time', type='number', value=10),
    html.Button('Сканировать', id='scan-button'),
    html.Div(id='output-table')
])

@app.callback(
    dash.dependencies.Output('output-table', 'children'),
    [dash.dependencies.Input('scan-button', 'n_clicks')],
    [dash.dependencies.State('start-range', 'value'),
     dash.dependencies.State('end-range', 'value'),
     dash.dependencies.State('scan-time', 'value')])
async def scan_devices(n_clicks, start_range, end_range, scan_time):
    device_names = ["TD_" + str(i) for i in range(start_range, end_range+1)]
    devices = {name: {'name': name, 'address': None, 'rssi': None, 'uuids': None} for name in device_names}
    table = html.Table([
        html.Thead(html.Tr([html.Th('Name'), html.Th('Address'), html.Th('RSSI'), html.Th('UUIDs')])),
        html.Tbody([html.Tr([
            html.Td(devices[name]['name']),
            html.Td(devices[name]['address']),
            html.Td(devices[name]['rssi']),
            html.Td(devices[name]['uuids'])
        ], style={'color': 'red'}) for name in devices])
    ])
    for i in range(scan_time):
        devices_found = await BleakScanner.discover()
        for d in devices_found:
            if d.name in devices:
                devices[d.name]['address'] = d.address
                devices[d.name]['rssi'] = d.rssi
                devices[d.name]['uuids'] = d.metadata['uuids']
                table.children[1].children[device_names.index(d.name)].style = {'color': 'green'}
    return table

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\Dukalis\AppData\Local\Temp\ipykernel_4780\1068401516.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Dukalis\AppData\Local\Temp\ipykernel_4780\1068401516.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [25/May/2023 12:01:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 12:01:52] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_9_3m1685005281.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 12:01:52] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_9_3m1685005281.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 12:01:52] "GET /_dash-component-suites/dash/deps/react@16.v2_9_3m1685005281.14.0.min.js H

C:\ProgramData\anaconda3\lib\site-packages\werkzeug\datastructures.py:1025: RuntimeWarning:

coroutine 'scan_devices' was never awaited

127.0.0.1 - - [25/May/2023 12:02:04] "POST /_dash-update-component HTTP/1.1" 500 -
